In [73]:
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 600)
from data_processing import punc_removal as pr
import us

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Import data

In [2]:
df = pd.read_csv('../cleaned_data.csv')

### Define column of importance

In [3]:
places = df['What_locations_do_you_WANT_to_live_in'].dropna()

### Explode data to work with

In [15]:
exploded_places = places.str.split(',').explode().reset_index(drop=True)

### Remove punctuation

In [21]:
cleaned = exploded_places.apply(pr)

### Remove all non-location words

In [22]:
words = pd.read_csv('data/dictionary.txt')

In [36]:
words = set(words['a'].values)

In [56]:
def word_removal(row):
    temp = [word for word in row.split() if word not in words]
    return ' '.join(temp)

In [96]:
removed = cleaned.apply(word_removal)

### Map State abbreviations to State names

In [137]:
#using imported "us" module
state_dict = us.states.mapping('abbr', 'name')
def abbr_mapper(row):
    if row in state_dict:
        return state_dict[row]
    else: return row

In [138]:
states = removed.apply(abbr_mapper)

### Now for the hard work....!

In [126]:
locations = {}

def collector(row):
    if len(row) < 1:
        pass
    else:
        if row in locations:
            locations[row] += 1
        else:
            locations[row] = 1
    return row

- San Diego

In [128]:
san_diego = states[states.str.contains('San Diego')].str.split('San Diego').explode().str.strip()

In [131]:
#san_diego.apply(collector)

In [130]:
locations

{'Southern CA': 1,
 'Southern California LA': 1,
 'California': 1,
 'New York': 1,
 'Dallas': 1,
 'Anywhere': 1,
 'Anywhere Southern California Ventura': 1,
 'San Francisco': 1,
 'Denver': 1,
 'Greater': 1,
 'Area Mid West': 1,
 'West Coast': 2,
 'Silicon Valley': 1,
 'NC Bay Area': 1,
 'New Orleans': 1,
 'Moving': 1,
 'Dec 2020': 1,
 'Irvine': 1}

In [141]:
states[states.str.contains('San Diego')] = 'San Diego'

- Remove all blank spaces in dataset

In [147]:
states = states[states != '']

- Abbreviation transformation

In [161]:
abbr = states.str.split().explode()

In [164]:
abbr = abbr[abbr.str.len() < 3]

In [168]:
#abbr.apply(abbr_mapper).apply(collector)

    - Clean out location data

In [179]:
'''bad_keys = []
for key, value in locations.items():
    if len(key) < 3:
        bad_keys.append(key)
for key in bad_keys:
    locations.pop(key, None)
'''

'bad_keys = []\nfor key, value in locations.items():\n    if len(key) < 3:\n        bad_keys.append(key)\nfor key in bad_keys:\n    locations.pop(key, None)\n'

In [195]:
bad_index = set(abbr.index)

In [199]:
locations

{'Southern CA': 1,
 'Southern California LA': 1,
 'California': 18,
 'New York': 4,
 'Dallas': 1,
 'Anywhere': 1,
 'Anywhere Southern California Ventura': 1,
 'San Francisco': 1,
 'Denver': 1,
 'Greater': 1,
 'Area Mid West': 1,
 'West Coast': 2,
 'Silicon Valley': 1,
 'NC Bay Area': 1,
 'New Orleans': 1,
 'Moving': 1,
 'Dec 2020': 1,
 'Irvine': 1,
 'Florida': 6,
 'Utah': 1,
 'District of Columbia': 36,
 'Virginia': 11,
 'Massachusetts': 3,
 'Arizona': 1,
 'New Hampshire': 4,
 'New Mexico': 1,
 'North Carolina': 17,
 'Texas': 11,
 'Montana': 1,
 'Tennessee': 3,
 'Pennsylvania': 1,
 'Illinois': 3,
 'Michigan': 2,
 'Colorado': 3,
 'Oregon': 4,
 'Washington': 3,
 'South Carolina': 2,
 'Georgia': 3,
 'Connecticut': 1,
 'West Virginia': 1}